In [5]:
import requests
import urllib
import pandas as pd 
from bs4 import BeautifulSoup
import json
import geopandas as gpd

In [9]:
#Load Daily data
page = requests.get("https://info.gesundheitsministerium.at/data/Trend.js")
page.content
soup = BeautifulSoup(page.content, 'html.parser')
a = [item for item in soup.children]
jsonStr = a[0].split("var dpTrend = ")[1].split(";")[0]
jsonData = json.loads(jsonStr)
df = pd.DataFrame(jsonData)
df.columns = ['date', 'cases']
df["date"] = df["date"]+".2020"
df.to_csv("austria_daily.csv",index = False)

In [10]:
#Load City Data 
import codecs
page = requests.get("https://info.gesundheitsministerium.at/data/Bezirke.js")
page.content
soup = BeautifulSoup(page.content, 'html.parser')
a = [item for item in soup.children]
jsonStr = a[0].split("var dpBezirke = ")[1].split(";")[0]
jsonData = json.loads(jsonStr)
df = pd.DataFrame(jsonData)
df.columns = ['name', 'cases']
gdf = gpd.read_file('STATISTIK_AUSTRIA_POLBEZ_20200101Polygon.shp',encoding="UTF-8")
from shapely import wkt
gdf_merged = pd.merge(df, gdf, on='name')
gdf_merged.head(5)
gdf_merged = gpd.GeoDataFrame(gdf_merged)
gdf_merged.plot()
gdf_merged.to_csv("austria_gdf")

KeyError: 'name'